## Mathematics for AI - Coursework Task 1

The first task tests your Python skills. You need to develop a simple game consisting of a rectangular grid (of size height x width ) where each cell has a random integer value between 0  and 9. An agent starts at the upper-left corner of the grid and must reach the lower-right corner of the grid as fast as possible. You can implement one of the two (or both, for no extra point) game modes:

• The time spent on a cell is the number on this cell

• The time spent on a cell is the absolute of the difference between the previous cell the agent was on and the current cell it is on.

In order to solve this problem, you will provide 3 algorithms:

• A first baseline of your choosing. E.g. it can be any search algorithm, or an algorithm using heuristics. It doesn’t have to perform fast or well, but should be better than random movements.

• Dijkstra's algorithm

• Ant colony optimization algorithm. You should describe the algorithms and compare them.

Are they always solving the problem? How long do they take depending on the size of the maze?

### Notes
- grid of size **heightxwidth**
    - each cell = random integer from 0-9
- **START** upper-left
- **GOAL** get to lower-right as fast as possible
- number on the cell = time spent on that cell

## Make Grid

In [2]:
# Import (limited) libraries
from random import seed
from random import randint
import numpy as np
import sys 

In [3]:
g = []
width = 3
height = 3
seed(6)

for row in range(height):
    g.append([])
    for column in range(width):
        g[row].append(randint(0,9))

In [4]:
# Convert to numpy.array
grid = np.array(g)
print(grid)

[[9 1 7]
 [4 0 0]
 [2 9 7]]


## Baseline Algorithm

In [5]:
class Baseline():
    
    def __init__(self, graph):
        self.graph = graph
    
    # Algorithm goes down to last row, then right to last column -> finish
    def algorithm(self):
        src = [0,0]
        current = src
        timeSpent = 0 # Total time spent by algorithm
        X = current[1]
        y = current[0]
        
        # Go down through rows
        while(X != (height-1)):
            X += 1
            timeSpent += self.graph[X,y]; # Value of field added to total
  
        # Go right through columns
        while(y != (width-1)):
            y += 1
            timeSpent += self.graph[X,y]; # Value of field added to total
            
        return timeSpent

In [6]:
base = Baseline(grid)
result = base.algorithm()
print(result)

22


**Description**:

- Assuming the agent can move one cell in 4 directions (LEFT, RIGHT, UP, DOWN)
- Check adjacent cells for the quickest cell, while ignoring already visited cells.
- Move to that cell
- repeat until the end has been reached

This algorithm does not neccesarily get the shortest path to the end, however its pathing is not random as it tries to move to the lowest valued cell adjacent to it (avoiding previously visited cells). 

A simple way that this algorithm could be improved:
When recording the value of the cell, it should be divided by a scalar factor (the closer it is to the goal the smaller the calculation will be); this will reduce the value of the cell that is closer to the end goal and favour it over moving in a direction away from the goal.

## Dijkstra's Algorithm

In [111]:
class Dijkstra:
    
    def __init__(self, grid, source, finish):
        self.grid = grid
        self.src = source
        self.finish = finish
    
    def get_neighbors(self, location):
        neighbors = []
        if (location[0] + 1) < (height):
            neighbors.append([location[0] + 1, location[1]])
                              
        if (location[0] - 1) >= 0:
            neighbors.append([location[0]-1, location[1]])
            
        if (location[1] + 1) < (width):
            neighbors.append([location[0], location[1] + 1])
            
        if (location[1] - 1) >= 0:
            neighbors.append([location[0], location[1] - 1])
        
        return neighbors
    
    def get_edges(self, location):
        neighbors = self.get_neighbors(location)
        edges = []
        
        for i in neighbors:
            edges.append(self.grid[i[0], i[1]])
            
        return edges
      
    def get_name(self, location):
        node_name = (location[0] * 100) + location[1]
        return str(node_name)
    
    def make_node(self, location):
        
        neighbors = self.get_neighbors(location)
        edges = self.get_edges(location)
        neighbor_names = []
        node_name = self.get_name(location)
        
        for i in neighbors:
            neighbor_names.append(self.get_name(i))
            
        neighbor_edges = dict(zip(neighbor_names, edges))
        
        return node_name, neighbor_edges
    
    def convert_grid(self):
        nodes = {}
        for i in range(self.finish[0]):
            for j in range(self.finish[1]):
                node = self.make_node([i,j])
                nodes[node[0]] = node[1]
        
        return nodes

In [136]:
src = [0,0]
finish = [height, width]

In [133]:
def shortestpath(graph,start,end,visited=[],distances={},predecessors={}):
    """Find the shortest path btw start & end nodes in a graph"""
    # detect if first time through, set current distance to zero
    if not visited: distances[start]=0
    # if we've found our end node, find the path to it, and return
    if start==end:
        path=[]
        while end != None:
            path.append(end)
            end=predecessors.get(end,None)
        return distances[start], path[::-1]
    # process neighbors as per algorithm, keep track of predecessors
    for neighbor in graph[start]:
        if neighbor not in visited:
            neighbordist = distances.get(neighbor,sys.maxsize)
            tentativedist = distances[start] + graph[start][neighbor]
            if tentativedist < neighbordist:
                distances[neighbor] = tentativedist
                predecessors[neighbor]=start
    # neighbors processed, now mark the current node as visited 
    visited.append(start)
    # finds the closest unvisited node to the start 
    unvisiteds = dict((k, distances.get(k,sys.maxsize)) for k in graph if k not in visited)
    closestnode = min(unvisiteds, key=unvisiteds.get)
    # now take the closest node and recurse, making it current 
    return shortestpath(graph,closestnode,end,visited,distances,predecessors)

In [144]:
s = (src[0] * 100) + src[1]
e = ((finish[0] * 100) + finish[1]) - 101

In [145]:
d = Dijkstra(grid, src, finish)
graph = d.convert_grid()

print (shortestpath(graph,str(s),str(e)))

(8, ['0', '1', '101', '102', '202'])


## Ant Colony Algorithm

- pheromones:     when ants travel they will leave some pheromones behind, the strenght of their pheromones will be calculated.
- vaporisation:   the pheromones will evaporate at a constant rate over time
- multiple ants do the this.
    - the pheromones increase the probability of the ants following that path.
- then the final ant takes the one with the highest pheromones.

